## Prepare train_data and test_data

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
X,y = datasets.load_breast_cancer(as_frame=True,return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=335)
train_data = pd.concat([X_train,y_train],axis=1)
eval_data = pd.concat([X_test,y_test],axis=1)

# Use HyperGBM to train model

In [3]:
from hypergbm import HyperGBM
from hypergbm.search_space import search_space_general
from hypernets.searchers.random_searcher import RandomSearcher

In [4]:
# instantiate related objects
searcher = RandomSearcher(search_space_general, optimize_direction='max')
hypergbm = HyperGBM(searcher, task='binary', reward_metric='accuracy')
# search
hypergbm.search(X_train, y_train, X_eval=None, y_eval=None,cv=True, max_trials=20)

08-17 07:09:48 I hypernets.c.meta_learner.py 22 - Initialize Meta Learner: dataset_id:ab373b8b2ef606325314215dd3595e3d
08-17 07:09:49 I hypergbm.hyper_gbm.py 308 - oof_scores:[{'accuracy': 0.9624060150375939}, {'accuracy': 0.9323308270676691}, {'accuracy': 0.9242424242424242}]
08-17 07:09:49 I hypernets.d.in_process_dispatcher.py 76 - Trial 1 done, reward: 0.9396984924623115, best_trial_no:1, best_reward:0.9396984924623115

08-17 07:09:50 I hypergbm.hyper_gbm.py 308 - oof_scores:[{'accuracy': 0.9624060150375939}, {'accuracy': 0.9323308270676691}, {'accuracy': 0.946969696969697}]
08-17 07:09:50 I hypernets.d.in_process_dispatcher.py 76 - Trial 2 done, reward: 0.9472361809045227, best_trial_no:2, best_reward:0.9472361809045227

08-17 07:09:50 I hypergbm.hyper_gbm.py 308 - oof_scores:[{'accuracy': 0.6165413533834586}, {'accuracy': 0.6090225563909775}, {'accuracy': 0.6136363636363636}]
08-17 07:09:50 I hypernets.d.in_process_dispatcher.py 76 - Trial 3 done, reward: 0.6130653266331658, best

In [5]:
# load best model
best_trial = hypergbm.get_best_trial()
estimator = hypergbm.load_estimator(best_trial.model_file)
best_trial

key,value
Trial No.,14
Reward,0.9673366834170855
Elapsed,0.1769425868988037
space.signature,f774870a60a8dd4f0a6138b43d5a62e2
space.vectors,"[0, 0, 0, 0, 390, 2, 2, 2, 0]"
0-estimator_options.hp_or,0
1-numeric_imputer_0.strategy,mean
2-numeric_scaler_optional_0.hp_opt,False
3-Module_LightGBMEstimator_1.boosting_type,gbdt
4-Module_LightGBMEstimator_1.num_leaves,390


In [6]:
# Scoring
from hypernets.tabular.metrics import calc_score
preds=estimator.predict(X_test)
proba=estimator.predict_proba(X_test)
scores = calc_score(y_test,preds,proba,task='binary', metrics=['auc','accuracy','f1','recall','precision'] )
scores

{'auc': 0.9812328349099787,
 'accuracy': 0.9590643274853801,
 'f1': 0.9691629955947135,
 'recall': 0.9734513274336283,
 'precision': 0.9649122807017544}